In [2]:
library(dplyr)

In [8]:
season <- seq(1999,2006)
games <- c(66,68,80,82,65,66,80,42)
success <- c(554,701,749,868,516,573,978,399)
attempts <- c(1183,1510,1597,1924,1178,1324,2173,845)
percents <- c(46.8,46.4,46.9,45.1,43.8,43.3,45,47.2)
fails <- attempts-success

data <- data.frame(Season=season,
                   Games=games,
                   Success=success,
                   Attempts=attempts,
                   Fails=fails,
                   Percents=percents)
data <- rbind(data,c(apply(data[,1:5],2,sum),round(mean(data[,6]),1)))
data[9,1] <- 'Total'
data


Season,Games,Success,Attempts,Fails,Percents
1999,66,554,1183,629,46.8
2000,68,701,1510,809,46.4
2001,80,749,1597,848,46.9
2002,82,868,1924,1056,45.1
2003,65,516,1178,662,43.8
2004,66,573,1324,751,43.3
2005,80,978,2173,1195,45.0
2006,42,399,845,446,47.2
Total,549,5338,11734,6396,45.6


# 1.Model
## Model : Logistic Regression  
슛의 성공 또는 실패 데이터이기 때문에 이항자료에 적합한 로지스틱 회귀모형을 사용할 것이다.  
따라서 로그오즈비($\theta$)에 대한 Inference를 하기로 한다.  
## Prior Distribution
이항분포와 Conjugate Prior인 베타분포를 Prior로 사용하도록 하자.  

In [9]:
cond_exp <- function(a,b) {
  a/(a+b)
}

cond_var <- function(a,b) {
  a*b/((a+b+1)*(a+b)^2)
}

## 1) Non-informative Prior
무정보 사전분포를 사용하기 위해 $\theta$~Beta(0.01,0.01)을 prior로 사용하자.

In [10]:
a1=0.01;b1=0.01 ##prior setting

M1 <- numeric(1)
V1 <- numeric(1)
for (i in 1:9) {
  M1[i] = cond_exp(data$Success[i]+a1, data$Fails[i]+b1)
  V1[i] = cond_var(data$Success[i]+a1, data$Fails[i]+b1)
}

data를 사용하여 각 년도별로 posterior mean과 posterior standard deviation을 구하는 코드로  
M벡터에는 각 년도별로 사후평균, V벡터에는 사후 표준편차가 구해져 있다.

In [11]:
res1 <- data.frame(Post_mean=round(M1*100,1),
                   Post_var=round(sqrt(V1)*100,1))
res1

Post_mean,Post_var
46.8,1.5
46.4,1.3
46.9,1.2
45.1,1.1
43.8,1.4
43.3,1.4
45.0,1.1
47.2,1.7
45.5,0.5


## 2) Improper Prior
$\theta$~Beta(0,0)가정을 통해 improper prior일 경우에 대해 알아보자

In [12]:
a2=0;b2=0
M2=numeric(1);V2=numeric(1)
for(i in 1:9) {
  M2[i]=cond_exp(data$Success[i]+a2, data$Fails[i]+b2)
  V2[i]=cond_var(data$Success[i]+a2, data$Fails[i]+b2)
}

res2 <- data.frame(Post_mean=round(M2*100,1),
                   Post_var=round(sqrt(V2)*100,1))
res2

Post_mean,Post_var
46.8,1.5
46.4,1.3
46.9,1.2
45.1,1.1
43.8,1.4
43.3,1.4
45.0,1.1
47.2,1.7
45.5,0.5


Beta(0.01,0.01)과 Beta(0,0)을 사용한 결과 차이가 없음을 알 수있다.  
간단하게 해석하면 사전분포로 Beta(0.01,0.01)을 사용한다는 것은 필드골의 성공 횟수에 a=0.01만큼 더하고, 실패 횟수에 b=0.01만큼 더해 보정한 다음 계산을 하는 것이기 때문에 성공,실패 횟수에 각각 0씩 보정하는것과 큰 차이가 없기 때문에 두 결과가 동일하게 나온다.

In [18]:
##Post_mean & Post_varinance function
post <- function(a,b) {
  M=numeric(1);V=numeric(1)
  for(i in 1:9) {
    M[i]=cond_exp(data$Success[i]+a, data$Fails[i]+b)
    V[i]=cond_var(data$Success[i]+a, data$Fails[i]+b)
  }
  R=data.frame(Post_mean=round(M*100,1),
               Post_var=round(sqrt(V)*100,1))
  return(R)
}

사전분포 Beta(a,b)를 사용했을 때의 사후분포의 평균, 표준편차를 구하는 함수를 만들었다.  
사전분포를 어떻게 주느냐에 따라 결과가 바뀌기 때문에 함수의 파라미터는 사전분포의 파라미터인 a,b로 설정

## 3) Improper Prior Beta(0,0) & Bayesian Update
2)에서 사용한 사전분포와 베이지안 접근을 통해 과거 데이터를 사용한 결과를 확인해 보자.

In [16]:
D=data
for(i in 2:nrow(data)) {
  D$Games[i]=D$Games[i-1]+data$Games[i]
  D$Success[i]=D$Success[i-1]+data$Success[i]
  D$Attempts[i]=D$Attempts[i-1]+data$Attempts[i]
  D$Fails[i]=D$Fails[i-1]+data$Fails[i]
  D$Percents[i]=D$Success[i]/D$Attempts[i]
}
D[1,6] <- D[1,3]/D[1,4]
D

Season,Games,Success,Attempts,Fails,Percents
1999,66,554,1183,629,0.4683009
2000,134,1255,2693,1438,0.4660230
2001,214,2004,4290,2286,0.4671329
2002,296,2872,6214,3342,0.4621822
2003,361,3388,7392,4004,0.4583333
2004,427,3961,8716,4755,0.4544516
2005,507,4939,10889,5950,0.4535770
2006,549,5338,11734,6396,0.4549173
Total,1098,10676,23468,12792,0.4549173


먼저 각 년도별로 성공, 총 시도횟수, 실패횟수를 누적하여 계산한 데이터는 위와같이 나온다.  
베이지안 업데이트를 사용하기 위해 새로운 데이터셋을 생성했기 때문에 새로운 post함수를 만들어 사용하자.

In [17]:
post_BU <- function(a,b) {
  M=numeric(1);V=numeric(1)
  for(i in 1:8) {
    M[i]=cond_exp(D$Success[i]+a, D$Fails[i]+b)
    V[i]=cond_var(D$Success[i]+a, D$Fails[i]+b)
  }
  R=data.frame(Post_mean=round(M*100,1),
               Post_var=round(sqrt(V)*100,1))
  return(R)
}

원 데이터의 마지막 행인 Total은 2006년도 데이터에 단지 2배만 된겄이기 때문에 사용하지 않기로 한다.  

In [19]:
R1 <- cbind(post(0,0)[1:8,]$Post_var, post_BU(0,0)$Post_var)
colnames(R1) <- c("Ordinal_Post_SD","Updated_Post_SD")
R1

Ordinal_Post_SD,Updated_Post_SD
1.5,1.5
1.3,1.0
1.2,0.8
1.1,0.6
1.4,0.6
1.4,0.5
1.1,0.5
1.7,0.5


R1의 Original_Post_SD는 베이지안 업데이트를 하지 않고 단지 사전분포를 가정한 결과를 보여주며  
Updated_Post_SD는 베이지안 업데이트를 통해 구한 사후분포의 표준편차를 보여준다.  

두 결과를 비교해 보면 업데이트를 사용한 결과의 분산이 훨씬 작게 나타나는 것을 볼 수 있다.  
Total행을 제외했기 때문에 총 8년간의 결과만 보여주고 있다.

## 4) Estimate p,o and theta
Matropolis-Hastings algorithm을 사용해 p,o 그리고 theta에 대해 샘플링을 해보도록 한다.  
$y$ ~ $Bin(n,p)$,  $o = p/(1-p)$,  $\theta$ = $logo$

### (1) Inference for $\theta$
$\theta,  o,  p$ 에 대해 MH알고리즘을 사용할 때 2006/2007년도의 데이터만 사용하기로 한다.  
Random-walk Metropolis를 사용해 사전분포를 정규분포로 주고 적절한 평균과 분산을 주기위해 해당 년도의 로그오즈비를 평균으로 가지고  
분산은 0.1,0.2,0.35,0.85 중 acceptance rate이 25~45%정도되는 분산을 선택하여 사용하자

In [21]:
##likelihood function
d <- data[8,]
mu=log((d$Percents/100) / (1-d$Percents/100))
mu=round(mu,2)
sig=c(0.1,0.2,0.35,0.85)
likelihood=function(theta) {
  exp(theta*d$Success) / ((1+exp(theta))^d$Attempts) *exp(-1/2*((theta-mu)/sig)^2)
}

$\theta$에 대한 Likelihood에 Prior를 곱한 값을 구하는 함수를 생성하여 MH에서 새로 생성된 샘플을 사용할 것인지 선택할 때 사용한다.